## RFInder: investigate RFI over timesteps set by the user


In [1]:
import os, string, sys
import numpy as np
import pyaml, json
from astropy.io import ascii
import logging
logger = logging.getLogger()
from IPython.display import HTML, display

import rfinder as rfinder

print '\t+------+\n\t  Done \n\t+------+'


Bad key "backed" on line 1 in
/Users/maccagni/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


	+------+
	  Done 
	+------+


### Load and modify parameter file

In [2]:
print '\t+------+\n\t RFInder \n\t Parameter File \n\t+------+'

rfi_par = rfinder.rfinder()

print '\t+------+\n\t Parameters Loaded \n\t+------+'

#print parameter file 
#print(json.dumps(rfi_par.cfg_par, indent=4, sort_keys=True))


INFO:rfinder:	 ... Reading parameter file ... 



	+------+
	 RFInder 
	 Parameter File 
	+------+
	+------+
	 Parameters Loaded 
	+------+


In [3]:
# Set working directory & filenames
#rfi_par.cfg_par['general']['workdir'] = '/home/maccagni/RFI/'

# Enable / disable modules
rfi_par.cfg_par['rfi']['enable'] = True
rfi_par.cfg_par['plots']['enable'] = True

#MSFILE
rfi_par.cfg_par['general']['msname'] = ['fa1_conc.ms']
rfi_par.cfg_par['general']['field'] = 1

rfi_par.set_dirs()

# CHUNKS
#create also tables of data binned in frequency and time
rfi_par.cfg_par['rfi']['chunks']['time_enable'] = True
rfi_par.cfg_par['rfi']['chunks']['time_step'] = 30 #in minutes

rfi_par.cfg_par['rfi']['chunks']['spw_enable'] = True
rfi_par.cfg_par['rfi']['chunks']['spw_width'] = 50 #MHz


#Options
#set separation in long and short baselines
rfi_par.cfg_par['rfi']['baselines_cut'] = 200 #m

#plotting options
rfi_par.cfg_par['plots']['long_short'] = True
rfi_par.cfg_par['plots']['plot_noise'] = 'flag'

print '\t+------+\n\t Parameters modified \n\t+------+'


	+------+
	 Parameters modified 
	+------+


### Investigate presence of RFI time step by time step

In [4]:
import rfi
import rfinder_plots as rfi_pl
import rfinder_stats as rfi_st
import rfinder_files as rfi_fl

import rfinder_plots as rfi_pl
from astropy import units as u
rfiST = rfi_st.rfi_stats()
rfi = rfi.rfi()
rfiPL = rfi_pl.rfi_plots()



#read info from MSfile 
#rfi.load_from_ms(rfi_par.cfg_par)
times,start,end = rfiST.time_chunk(rfi_par.cfg_par)
for i in xrange(0,len(times)-1):
#for i in xrange(0,2):
    
    timez = [times[i],times[i+1]] 
    rfi.load_from_ms(rfi_par.cfg_par,timez)

    #sort visibilities by baseline lenght
    rfi.baselines_from_ms(rfi_par.cfg_par)

    #flag bad antennas (from configuration file)
    datas = rfi.priors_flag(rfi_par.cfg_par)

    #find rfi above threshold
    rfi.find_rfi(datas,rfi_par.cfg_par,i)
    rfi_fl.rfi_frequency(rfi_par.cfg_par,i)

    
rfiPL.plot_altaz(rfi_par.cfg_par,i)

print '\t+------+\n\t Done \n\t+------+'

INFO:rfinder_stats:	 ...  Observing time Info ... 



Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows


INFO:rfinder_stats:	 Start date: 17Sep07:22:28:15
INFO:rfinder_stats:	 End date  : 17Sep08:07:58:31 


INFO:rfi:	 ... Field, Antenna & Bandwidth Info ...

INFO:rfi:	Field with name Fornax (Field ID = 1)
INFO:rfi:	Total number of antennas:	16
INFO:rfi:	Antenna names:		[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
INFO:rfi:	Channel Width [kHz]:	208.984375
INFO:rfi:	Start         [GHz]:	1.29716601563
INFO:rfi:	End           [GHz]:	1.44345507812

INFO:rfinder_stats:	 ...  Observing time Info ... 



Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/FIELD: 9 columns, 4 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/ANTENNA: 8 columns, 16 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows


INFO:rfinder_stats:	 Start date: 17Sep07:22:28:15
INFO:rfinder_stats:	 End date  : 17Sep08:07:58:31 


INFO:rfinder_stats:	 ... Altitude/Azimuth info ... 



...

Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/FIELD: 9 columns, 4 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/ANTENNA: 8 columns, 16 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows


INFO:rfinder_stats:	 ...  Predicting natural r.m.s. ... 

INFO:rfinder_stats:	 Assumptions on meerkat telescope
INFO:rfinder_stats:		Dish diameter = 13.5 m
INFO:rfinder_stats:		 ... and SEFD = 578.710952 Jy
INFO:rfinder_stats:		 ... and Tsys = 30.0 K
INFO:rfinder_stats:	 Properties of observation
INFO:rfinder_stats:		 Total number of baselines = 120.0
INFO:rfinder_stats:		 Total number of channels = 701
INFO:rfinder_stats:		 Observing time on source = 0.37540 h (2 polarisations)

INFO:rfinder_stats:	 Stokes I natural r.m.s.       = 1.572e+00 mJy/b 
INFO:rfinder_stats:	 ... Natural r.m.s. predicted ... 

INFO:rfi:	 ... info from MS file loaded  


INFO:rfi:	 ... Sorting baselines ... 

INFO:rfinder_stats:	 ... Baseline info ... 

INFO:rfinder_stats:		 Maximum baseline length     m = 5204.0
INFO:rfinder_stats:		 Minimum baseline length     m = 29.0
INFO:rfinder_stats:		 Total number of baselines     = 120.0
INFO:rfinder_stats:		 Number of baselines < 200 m = 4
INFO:rfinder_stats:		 Numbe

Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/FIELD: 9 columns, 4 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/ANTENNA: 8 columns, 16 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms/SPECTRAL_WINDOW: 14 columns, 1 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows
Successful readonly open of default-locked table /Users/maccagni/Projects/RFI/rfinder_test/fa1_conc.ms: 27 columns, 475920 rows


INFO:rfinder_files:	 ... Wrtiting RFI by frequency and baseline ...
INFO:rfinder_files:	 ... RFI on fits file ...

INFO:rfi:	 ... RFI found ... 


INFO:rfinder_files:	 ... RFI table saved ...

INFO:rfinder_plots:	 ... Plotting Alt/Az for binned dataset ... 



KeyError: u'ytick.right is not a valid rc parameter.See rcParams.keys() for a list of valid parameters.'